# CONFLUENCE Tutorial 02c — Basin-Scale Workflow (Bow River at Banff, Elevation-Based Distributed)

## Introduction

This tutorial demonstrates the most spatially-detailed modeling approach: elevation-based HRU discretization. Building on Tutorials 02a (lumped) and 02b (semi-distributed), we now subdivide each GRU into elevation bands that capture altitudinal controls on mountain hydrology.

Elevation-based discretization is critical in mountain watersheds where temperature and precipitation vary systematically with altitude. By stratifying each sub-basin into elevation bands, we better represent snowpack dynamics, seasonal timing differences, and orographic effects.

The key configuration parameter is `ELEVATION_BAND_SIZE`, which controls the vertical resolution (e.g., 100m bands). Smaller bands increase spatial detail but add computational cost. This approach maintains the validated GRU structure from Tutorial 02b while adding vertical stratification.

For the **Bow River at Banff** (elevation range: 1,384–3,400 m), elevation bands capture the transition from low-elevation rain-dominated zones to high-elevation snow-dominated headwaters, improving simulation of spring freshet timing and runoff generation.


# Step 1 — Configuration and data reuse

We configure elevation-based discretization and reuse data from Tutorial 02b where possible.

In [ ]:
# Step 1 — Elevation-based configuration with data reuse

from pathlib import Path
import yaml
import shutil
import sys
sys.path.append(str(Path("../..").resolve()))
from CONFLUENCE import CONFLUENCE

# Define directories
CONFLUENCE_CODE_DIR = Path("../..").resolve()
CONFLUENCE_DATA_DIR = Path("/path/to/CONFLUENCE_data").resolve()

# Load template
config_template = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_template.yaml'
with open(config_template, 'r') as f:
    config = yaml.safe_load(f)

# === Modify for elevation-based distributed ===
config['CONFLUENCE_CODE_DIR'] = str(CONFLUENCE_CODE_DIR)
config['CONFLUENCE_DATA_DIR'] = str(CONFLUENCE_DATA_DIR)
config['DOMAIN_NAME'] = 'Bow_at_Banff_elevation'
config['EXPERIMENT_ID'] = 'run_1'
config['POUR_POINT_COORDS'] = '51.1722/-115.5717'

# Elevation-based discretization
config['DOMAIN_DEFINITION_METHOD'] = 'delineate'
config['STREAM_THRESHOLD'] = 5000  # Same as 02b
config['DOMAIN_DISCRETIZATION'] = 'elevation'  # Key change
config['ELEVATION_BAND_SIZE'] = 100  # 100m elevation bands

config['HYDROLOGICAL_MODEL'] = 'SUMMA'
config['ROUTING_MODEL'] = 'mizuRoute'

config['EXPERIMENT_TIME_START'] = '2011-01-01 01:00'
config['EXPERIMENT_TIME_END'] = '2018-12-31 23:00'
config['CALIBRATION_PERIOD'] = '2011-01-01, 2015-12-31'
config['EVALUATION_PERIOD'] = '2016-01-01, 2018-12-31'
config['SPINUP_PERIOD'] = '2011-01-01, 2011-12-31'
config['STATION_ID'] = '05BB001'
config['DOWNLOAD_WSC_DATA'] = True

# Save configuration
config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_elevation_distributed.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, sort_keys=False)

print(f"✅ Configuration saved: {config_path}")

# === Data reuse from Tutorial 02b ===
semi_dist_domain = 'Bow_at_Banff_semi_distributed'
semi_dist_data_dir = CONFLUENCE_DATA_DIR / f'domain_{semi_dist_domain}'

def copy_with_name_adaptation(src, dst, old_name, new_name):
    """Copy directory and adapt filenames"""
    if not src.exists():
        return False
    dst.parent.mkdir(parents=True, exist_ok=True)
    if src.is_file():
        shutil.copy2(src, dst)
        return True
    shutil.copytree(src, dst, dirs_exist_ok=True)
    for file in dst.rglob('*'):
        if file.is_file() and old_name in file.name:
            new_file = file.parent / file.name.replace(old_name, new_name)
            file.rename(new_file)
    return True

# Initialize CONFLUENCE
confluence = CONFLUENCE(config_path)
project_dir = confluence.managers['project'].setup_project()

if semi_dist_data_dir.exists():
    print(f"\n📋 Reusing data from Tutorial 02b: {semi_dist_data_dir}")
    
    reusable_data = {
        'Elevation': semi_dist_data_dir / 'attributes' / 'elevation',
        'Land Cover': semi_dist_data_dir / 'attributes' / 'land_cover',
        'Soils': semi_dist_data_dir / 'attributes' / 'soils',
        'Forcing': semi_dist_data_dir / 'forcing' / 'raw_data',
        'Stream Network': semi_dist_data_dir / 'shapefiles' / 'river_network',
        'GRUs': semi_dist_data_dir / 'shapefiles' / 'river_basins',
        'Streamflow': semi_dist_data_dir / 'observations' / 'streamflow'
    }
    
    for data_type, src_path in reusable_data.items():
        if src_path.exists():
            rel_path = src_path.relative_to(semi_dist_data_dir)
            dst_path = project_dir / rel_path
            success = copy_with_name_adaptation(src_path, dst_path, semi_dist_domain, config['DOMAIN_NAME'])
            if success:
                print(f"   ✅ {data_type}: Copied")
        else:
            print(f"   📋 {data_type}: Not found")
else:
    print(f"\n⚠️  No data from Tutorial 02b found. Will acquire fresh data.")

# Create pour point
pour_point_path = confluence.managers['project'].create_pour_point()
print(f"\n✅ Project structure created at: {project_dir}")

## Step 2 — Elevation-based discretization

Subdivide each GRU from Tutorial 02b into elevation bands for vertical stratification.

### Step 2a — Attribute check

Verify DEM and GRU availability from data reuse.

In [ ]:
# Step 2a — DEM and GRU availability check
dem_path = project_dir / 'attributes' / 'elevation' / 'dem'
gru_path = project_dir / 'shapefiles' / 'river_basins'

if not dem_path.exists() or not gru_path.exists():
    print("   Required data not found, acquiring...")
    # If using MAF supported HPC, uncomment the lines below
    # confluence.managers['data'].acquire_attributes()
    # confluence.managers['domain'].define_domain()
    print("✅ Geospatial data acquired")
else:
    print("✅ DEM and GRU data available from previous workflow")

### Step 2b — Elevation band creation

Create HRUs by intersecting GRUs with elevation bands.

In [ ]:
# Step 2b — Elevation-based HRU discretization
hru_path = confluence.managers['domain'].discretize_domain()
print("✅ Elevation-based HRU discretization complete")

### Step 2c — Elevation structure visualization

Visualize the elevation-stratified HRU structure.

In [ ]:
# Step 2c — Elevation band visualization

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

# Load HRUs with elevation information
hru_file = project_dir / 'shapefiles' / 'catchment' / f"{config['DOMAIN_NAME']}_HRUs_elevation.shp"

if hru_file.exists():
    hru_gdf = gpd.read_file(str(hru_file))
    print(f"Number of elevation-based HRUs: {len(hru_gdf)}")
    
    # Calculate elevation statistics
    if 'elev_mean' in hru_gdf.columns:
        elev_col = 'elev_mean'
    elif 'elevation' in hru_gdf.columns:
        elev_col = 'elevation'
    else:
        elev_col = hru_gdf.select_dtypes(include=[np.number]).columns[0]
    
    print(f"Elevation range: {hru_gdf[elev_col].min():.0f} - {hru_gdf[elev_col].max():.0f} m")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Spatial map colored by elevation
    hru_gdf.plot(column=elev_col, cmap='terrain', ax=axes[0], 
                 legend=True, legend_kwds={'label': 'Elevation (m)'})
    
    pour_point_gdf = gpd.read_file(pour_point_path)
    pour_point_gdf.plot(ax=axes[0], color='red', markersize=150, marker='*', label='Pour Point')
    
    axes[0].set_title(f'Elevation-Based HRU Distribution\n{len(hru_gdf)} HRUs', fontweight='bold')
    axes[0].set_xlabel('Longitude')
    axes[0].set_ylabel('Latitude')
    axes[0].legend()
    
    # Elevation distribution histogram
    axes[1].hist(hru_gdf[elev_col], bins=20, color='steelblue', edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Elevation (m)')
    axes[1].set_ylabel('Number of HRUs')
    axes[1].set_title('HRU Elevation Distribution', fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  HRU shapefile not found")

## Step 3 — Data preprocessing

Process forcing and observation data for elevation-stratified HRUs.

In [ ]:
# Step 3a — Streamflow observations
# If using MAF supported HPC, uncomment the line below
# confluence.managers['data'].process_observed_data()
print("✅ Streamflow data processing complete")

In [ ]:
# Step 3b — Forcing data
# If using MAF supported HPC, uncomment the line below
# confluence.managers['data'].acquire_forcings()
print("✅ Forcing acquisition complete")

In [ ]:
# Step 3c — Model-agnostic preprocessing
confluence.managers['data'].run_model_agnostic_preprocessing()
print("✅ Model-agnostic preprocessing complete")

## Step 4 — Model execution

Configure and run SUMMA-mizuRoute with elevation-stratified HRUs.

In [ ]:
# Step 4a — Model configuration
confluence.managers['model'].preprocess_models()
print("✅ Elevation-based model configuration complete")

In [ ]:
# Step 4b — Model execution
print(f"Running {config['HYDROLOGICAL_MODEL']} with {config['ROUTING_MODEL']} ({len(hru_gdf)} elevation-based HRUs)...")
confluence.managers['model'].run_models()
print("✅ Elevation-based distributed simulation complete")

## Step 5 — Evaluation

Compare elevation-based results against observations and previous approaches.

In [ ]:
# Step 5 — Elevation-based evaluation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

# Load observed streamflow
obs_path = project_dir / "observations" / "streamflow" / "preprocessed" / f"{config['DOMAIN_NAME']}_streamflow_processed.csv"
obs_df = pd.read_csv(obs_path, parse_dates=['datetime'])
obs_df.set_index('datetime', inplace=True)

# Load simulated streamflow
routing_dir = project_dir / "simulations" / config['EXPERIMENT_ID'] / "mizuRoute"
sim_files = list(routing_dir.glob('*_routed.nc'))
if not sim_files:
    raise FileNotFoundError(f"No routed streamflow in: {routing_dir}")

sim_ds = xr.open_dataset(sim_files[0])
sim_df = sim_ds['IRFroutedRunoff'].to_dataframe().reset_index()
sim_df = sim_df.rename(columns={'time': 'datetime', 'IRFroutedRunoff': 'discharge_sim'})
sim_df.set_index('datetime', inplace=True)

# Merge and align
eval_df = obs_df.join(sim_df, how='inner')
obs_valid = eval_df['discharge_obs'].dropna()
sim_valid = eval_df.loc[obs_valid.index, 'discharge_sim']

# Metrics
def nse(obs, sim):
    return float(1 - np.sum((obs - sim)**2) / np.sum((obs - obs.mean())**2))

def kge(obs, sim):
    r = obs.corr(sim)
    alpha = sim.std() / obs.std()
    beta = sim.mean() / obs.mean()
    return float(1 - np.sqrt((r-1)**2 + (alpha-1)**2 + (beta-1)**2))

def pbias(obs, sim):
    return float(100 * (sim.sum() - obs.sum()) / obs.sum())

nse_val = round(nse(obs_valid, sim_valid), 3)
kge_val = round(kge(obs_valid, sim_valid), 3)
pbias_val = round(pbias(obs_valid, sim_valid), 1)

print(f"Performance Metrics:")
print(f"  NSE: {nse_val}")
print(f"  KGE: {kge_val}")
print(f"  PBIAS: {pbias_val}%")
print(f"  Number of HRUs: {len(hru_gdf)}")

# Visualization - compact 2x2 layout
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time series
axes[0, 0].plot(obs_valid.index, obs_valid.values, 'b-', label='Observed', linewidth=1.2, alpha=0.7)
axes[0, 0].plot(sim_valid.index, sim_valid.values, 'r-', label=f'Elevation-Based ({len(hru_gdf)} HRUs)', 
                linewidth=1.2, alpha=0.7)
axes[0, 0].set_ylabel('Discharge (m³/s)')
axes[0, 0].set_title('Elevation-Based Streamflow')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].text(0.02, 0.95, f"NSE: {nse_val}\nKGE: {kge_val}\nBias: {pbias_val}%\nHRUs: {len(hru_gdf)}",
                transform=axes[0, 0].transAxes, verticalalignment='top',
                bbox=dict(facecolor='white', alpha=0.8), fontsize=9)

# Scatter
axes[0, 1].scatter(obs_valid, sim_valid, alpha=0.5, s=10, c='coral')
max_val = max(obs_valid.max(), sim_valid.max())
axes[0, 1].plot([0, max_val], [0, max_val], 'k--', alpha=0.5)
axes[0, 1].set_xlabel('Observed (m³/s)')
axes[0, 1].set_ylabel('Simulated (m³/s)')
axes[0, 1].set_title('Observed vs Simulated')
axes[0, 1].grid(True, alpha=0.3)

# Monthly climatology
monthly_obs = obs_valid.groupby(obs_valid.index.month).mean()
monthly_sim = sim_valid.groupby(sim_valid.index.month).mean()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
axes[1, 0].plot(monthly_obs.index, monthly_obs.values, 'b-o', label='Observed', markersize=6)
axes[1, 0].plot(monthly_sim.index, monthly_sim.values, 'r-o', label='Simulated', markersize=6)
axes[1, 0].set_xticks(range(1, 13))
axes[1, 0].set_xticklabels(month_names)
axes[1, 0].set_ylabel('Mean Discharge (m³/s)')
axes[1, 0].set_title('Seasonal Flow Regime')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Tutorial progression comparison
tutorial_data = {
    'Tutorial': ['02a\n(Lumped)', '02b\n(Semi-Dist)', '02c\n(Elevation)'],
    'Units': [1, 'N/A', len(hru_gdf)],  # Update with actual 02b GRU count if available
    'NSE': [0.65, 0.72, nse_val]  # Update with actual values from previous tutorials
}

x_pos = np.arange(len(tutorial_data['Tutorial']))
bars = axes[1, 1].bar(x_pos, tutorial_data['NSE'], 
                       color=['lightblue', 'lightgreen', 'lightcoral'], 
                       alpha=0.7, edgecolor='navy')
axes[1, 1].set_xlabel('Tutorial Progression')
axes[1, 1].set_ylabel('Nash-Sutcliffe Efficiency')
axes[1, 1].set_title('Performance vs Complexity')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(tutorial_data['Tutorial'])
axes[1, 1].set_ylim(0, 1)
axes[1, 1].grid(True, alpha=0.3, axis='y')

for i, (bar, units) in enumerate(zip(bars, tutorial_data['Units'])):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                    f'{height:.3f}\n({units} units)',
                    ha='center', va='bottom', fontsize=9)

plt.suptitle(f'Elevation-Based Evaluation — {config["DOMAIN_NAME"]} ({len(hru_gdf)} HRUs)',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✅ Elevation-based evaluation complete")

## Summary

This tutorial demonstrated the most spatially-detailed modeling approach in the CONFLUENCE basin-scale series. Elevation-based HRU discretization captures critical altitudinal controls on mountain hydrology by stratifying each sub-basin into elevation bands.

Key achievements:
- Elevation band creation within validated GRU structure
- Vertical stratification of temperature and precipitation
- Improved representation of snowpack dynamics across elevation gradients
- Efficient data reuse from Tutorial 02b

The progression from lumped (02a) → semi-distributed (02b) → elevation-based (02c) demonstrates CONFLUENCE's flexibility in managing spatial complexity. Each approach has its place:
- **Lumped**: Calibration, uncertainty analysis, baseline performance
- **Semi-distributed**: Spatial process attribution, moderate computational cost
- **Elevation-based**: Maximum detail for mountain systems, highest computational cost

This completes the basin-scale modeling tutorial series, establishing the foundation for large-scale distributed applications and operational hydrological forecasting.